In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from base_module import (
    get_preprocessed_data,
    train_and_evaluate,
    save_model_results,
    SVM_PARAMS,
    RANDOM_STATE
)
import time

ALGORITHM_NAME = "L1_Lasso_SVM"
# !! IMPORTANT: Update this path to your local file path !!
DATA_PATH = r'D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv'

print("="*70)
print(f"FEATURE SELECTION: {ALGORITHM_NAME}")
print("="*70)

FEATURE SELECTION: L1_Lasso_SVM


In [2]:
print("\n--- Loading and Preprocessing Data ---")

X_train_encoded, X_test_encoded, y_train, y_test, \
    all_encoded_features, all_original_features, feature_mapping = \
    get_preprocessed_data(DATA_PATH)

print(f"\nStarting with {len(all_encoded_features)} encoded features")


--- Loading and Preprocessing Data ---
Loading data from D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv...


d:\ACTUAL STUDY MATERIAL\IPD\src\Notebook\base_module.py:32: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, na_values='?')


Cleaned data shape: (98053, 43)
Engineering features...
Engineered data shape: (98053, 41)

Preprocessing data...
Ordinal Encoding 31 features...
Encoded features: 40
  - Numeric: 9
  - Categorical (Ordinal): 31

Starting with 40 encoded features


In [3]:
print(f"\n--- Applying {ALGORITHM_NAME} (SelectFromModel) ---")
start_time = time.time()

# --- 1. Define the Estimator ---
# We use LinearSVC with L1 penalty. L1 (Lasso) is known for
# pushing coefficients of unimportant features to exactly zero.
# We must use 'dual=False' for L1 penalty.
estimator = LinearSVC(
    penalty='l1',
    C=0.05,
    dual=False,
    random_state=RANDOM_STATE,
    max_iter=5000
)

# --- 2. Fit the L1 model ---
# We fit this *once* on the full training data.
print("Fitting L1 model to find feature importances...")
estimator.fit(X_train_encoded, y_train)

# --- 3. Initialize SelectFromModel ---
# This selector will simply "select" all features that have a
# non-zero coefficient. The 'threshold="1e-5"' is safer than '0'.
selector = SelectFromModel(estimator, prefit=True, threshold=1e-5)

elapsed = (time.time() - start_time) / 60
print(f"✓ L1 model fitting complete in {elapsed:.2f} minutes")

# --- 4. Get Results ---
X_train_selected = selector.transform(X_train_encoded)
X_test_selected = selector.transform(X_test_encoded)

selected_mask = selector.get_support()
selected_features = [all_encoded_features[i] for i, selected in enumerate(selected_mask) if selected]

print(f"\nFeature Selection Results:")
print(f"  Selected: {len(selected_features)} features")
print(f"  Eliminated: {len(all_encoded_features) - len(selected_features)} features")


--- Applying L1_Lasso_SVM (SelectFromModel) ---
Fitting L1 model to find feature importances...
✓ L1 model fitting complete in 0.30 minutes

Feature Selection Results:
  Selected: 28 features
  Eliminated: 12 features


d:\ACTUAL STUDY MATERIAL\IPD\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [4]:
print(f"\n--- Training Model on {ALGORITHM_NAME}-Selected Features ---")

# We train the *final* model using the standard SVM_PARAMS 
# (which use L2 penalty) for a fair comparison.
clf, y_pred, report = train_and_evaluate(
    X_train_selected, X_test_selected,
    y_train, y_test,
    **SVM_PARAMS
)

print(f"\nModel Performance:")
print(f"  Accuracy: {report['accuracy']:.4f}")
print(f"  Precision (class 0): {report['0']['precision']:.4f}")
print(f"  Recall (class 0): {report['0']['recall']:.4f}")
print(f"  F1-score (class 0): {report['0']['f1-score']:.4f}")
print(f"  Precision (class 1): {report['1']['precision']:.4f}")
print(f"  Recall (class 1): {report['1']['recall']:.4f}")
print(f"  F1-score (class 1): {report['1']['f1-score']:.4f}")


--- Training Model on L1_Lasso_SVM-Selected Features ---

Training SVM classifier...


d:\ACTUAL STUDY MATERIAL\IPD\.venv\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Evaluating...

Model Performance:
  Accuracy: 0.4066
  Precision (class 0): 0.8595
  Recall (class 0): 0.3958
  F1-score (class 0): 0.5420
  Precision (class 1): 0.0937
  Recall (class 1): 0.4912
  F1-score (class 1): 0.1574


In [5]:
print(f"\n--- Saving Results ---")

save_model_results(
    algorithm_name=ALGORITHM_NAME,
    selected_encoded_features=selected_features,
    all_encoded_features=all_encoded_features,
    all_original_features=all_original_features,
    feature_mapping=feature_mapping,
    report=report,
    results_csv='study_results.csv'
)

print("\n" + "="*70)
print(f"{ALGORITHM_NAME} COMPLETE")
print("="*70)


--- Saving Results ---

Saving results for: L1_Lasso_SVM
✓ Appended to D:\ACTUAL STUDY MATERIAL\IPD\src\Results\study_results.csv
✓ Saved to D:\ACTUAL STUDY MATERIAL\IPD\src\Results\l1_lasso_svm_results.json

  Original: 28/40 features
  Encoded:  28/40 features
  Accuracy: 0.4066
  F1 (class 1): 0.1574

L1_Lasso_SVM COMPLETE
